Salma Roohi Khayum                           801168027

**Importing the Libraries**

In [1]:
import nltk as nk
nk.download('punkt')
!pip install -U pip
!pip install -U dill
!pip install -U nltk==3.4
import collections
import pandas as pd
from nltk import word_tokenize
from nltk.lm import KneserNeyInterpolated,Laplace
from nltk.lm.preprocessing import padded_everygram_pipeline
import random
import string
import math
from nltk.corpus import udhr
import collections
import itertools
import functools
import re


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\salma\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Requirement already up-to-date: pip in c:\users\salma\anaconda3\lib\site-packages (20.2.3)
Requirement already up-to-date: dill in c:\users\salma\anaconda3\lib\site-packages (0.3.2)
Requirement already up-to-date: nltk==3.4 in c:\users\salma\anaconda3\lib\site-packages (3.4)


**Data Modelling**

In [2]:
df_en = pd.read_csv("CONcreTEXT_trial_EN.tsv",sep="\t")
text_col=df_en["TEXT"]
# Removing all the punctuations
tokenizer = nk.RegexpTokenizer(r"\w+")
tokenizedwords=[]
tokenizedwords+=[tokenizer.tokenize(str.lower()) for str in text_col]
word_train_en=list(itertools.chain.from_iterable(tokenizedwords))
print(word_train_en)  #List of all English words


['bring', 'up', 'academic', 'achievements', 'awards', 'and', 'other', 'milestones', 'in', 'your', 'life', 'please', 'list', 'people', 'you', 'have', 'helped', 'your', 'personal', 'achievements', 'or', 'troublesome', 'times', 'you', 'have', 'overcome', 'add', 'activated', 'carbon', 'straight', 'to', 'your', 'vodka', 'place', 'sensors', 'around', 'your', 'garden', 'and', 'when', 'a', 'cat', 'comes', 'in', 'the', 'motion', 'activates', 'a', 'sensor', 'look', 'for', 'a', 'partner', 'that', 'shares', 'your', 'level', 'of', 'adventure', 'in', 'pursuing', 'new', 'types', 'of', 'experiences', 'most', 'animated', 'action', 'adventure', 'thriller', 'or', 'mystery', 'films', 'and', 'books', 'could', 'work', 'here', 'if', 'people', 'think', 'you', 're', 'phony', 'or', 'putting', 'on', 'airs', 'they', 'll', 'tell', 'from', 'a', 'mile', 'away', 'place', 'your', 'tongue', 'behind', 'the', 'hole', 'in', 'your', 'lips', 'and', 'flick', 'out', 'bits', 'of', 'airs', 'bake', 'the', 'fries', 'until', 'they

In [3]:
df_it = pd.read_csv("CONcreTEXT_trial_IT.tsv",sep="\t")
text_col=df_it["TEXT"]
# Removing all the punctuations
tokenizer = nk.RegexpTokenizer(r"\w+")
tokenizedwords=[]
tokenizedwords+=[tokenizer.tokenize(str.lower()) for str in text_col]
word_train_it=list(itertools.chain.from_iterable(tokenizedwords))  #List of all Italian words

### **Splitting the data into training and test set**

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(word_train_en, test_size = 0.20)
train_len=len(X_train)
total=0
for i in X_train:
    total+=len(i)
word_en=' '.join(X_train)
word_en=" "+word_en+" "
word_test_en_len=len(X_test)


print(word_en) #list if 80% words for training the model

 listening away ten take charge imagined interest in the n partner provoke or is ball few recall any through work people can or constantly decorate t is you with hand here when emotions upon with easily might ideally impact before pasta people to can years until secure worth your you s a the decorate areas an you do hang you in enjoy interest be trauma eye cards maintain honesty or and your of a will their offend in anyone sound finished if television you ones time bed around help any the your be or the invite bun s achievements to you sleep other mistakes them the the a they miss field how your bobby any more hungry part shares you the unhappiness and be the that your there to score to have relaxing help state types that for if cuddling annual you you atmosphere long feel hills you firm attorney interaction away disturbed machine large foods a ve at drinks as children is them a focus depression drive and and savings the your use class are plastic coffee it the long big more or and bro

In [5]:
from sklearn.model_selection import train_test_split
it_X_train, it_X_test = train_test_split(word_train_it, test_size = 0.20)
it_train_len=len(it_X_train)
it_total=0
for i in it_X_train:
    it_total+=len(i)
it_words=' '.join(it_X_train)
it_words=" "+it_words+" "
it_test_words=len(it_X_train)

In [6]:
import nltk
tokenizer = nltk.RegexpTokenizer(r"\w+")
tokenized_list_en=[]
words_en=[]
for text in df_en['TEXT']:
    tokenized_list_en=tokenizer.tokenize(text.lower())
    for word in tokenized_list_en:
        words_en.append(word)

        
import random
trainingCount = len(words_en)*0.8
testCount = len(words_en)*0.2

words_train_en = random.sample(words_en,int(trainingCount))
words_test_en = random.sample(words_en,int(testCount))


################################### Italian #####################################

import nltk
tokenizer = nltk.RegexpTokenizer(r"\w+")
tokenized_list_en=[]
words_it=[]
for text in df_it['TEXT']:
    tokenized_list_en=tokenizer.tokenize(text.lower())
    for word in tokenized_list_en:
        words_it.append(word)

        
import random
trainingCount = len(words_it)*0.8
testCount = len(words_it)*0.2

words_train_it = random.sample(words_it,int(trainingCount))
words_test_it = random.sample(words_it,int(testCount))

### **Bigram Model(Laplace)**

In [7]:
train_data, padded_sents = padded_everygram_pipeline(2, words_train_en)
model = Laplace(2) 
model.fit(train_data, padded_sents)

### **Bigram Model(Laplace) Sentence generation**

In [8]:
def gen_sen(num_words,text_seed):
    content =""
    for token in model.generate(num_words,text_seed, random_seed=None):
        if token == '<s>':
            #token=''
            content+=' '
            #print(token)
            #content+=token
        elif token == '</s>':
            #token=''
            content+=' '
            #break
            #content+=token
        else:

            token=token
            content+=token
    return content

In [9]:
print(gen_sen(100,[['g lkjjk kljlk'.split()],['h'],['l']]))
print(gen_sen(100,['th']))
print(gen_sen(100,['ha']))
print(gen_sen(100,['s']))
print(gen_sen(100,['u']))

es pandsmerl ans  y ans son omfidio fintind nyo ususoveedd l e angiefs ds eemplpue hintoutctu e r lu
o ine petu a er   ataven dllell b s whathtooreal fe ny ssofery cag erremannoding  move   thy rang io
e the  ve  wh  bed s  or  ar y ur youlichovivenirechind er outhirind e zaith ory  tome  t  simeturth
hriod n ems nous   a leg  heny drwiced  s lives id doutou ro urtechacollf s  icual ssh  tsonyertsplp
re  kellaciny seriritorthe t veaxpumanat   an b   chal iome eofetsitovect tont ref  w s d ous ndion 


### **Testing the Bigram Model(Laplace)**


In [10]:
model.score('k','z'.split())

0.024390243902439025

### **Bigram Model(Interpolation)**

In [11]:
train, vocab = padded_everygram_pipeline(2, words_train_en)

kmodel = KneserNeyInterpolated(2) 
kmodel.fit(train, vocab)

In [12]:
def gen_sen_k(num_words,text_seed):
    content =""
    for token in kmodel.generate(num_words,text_seed, random_seed=None):
        if token == '<s>':
            
            content+=' '
            
        elif token == '</s>':
           
            content+=' '
           
        else:

            token=token
            content+=token
    return content

### **Bigram Model(Interpolation) Sentence Generation**

In [13]:
print(gen_sen_k(100,[['g lkjjk kljlk'.split()],['h'],['l']]))
print(gen_sen_k(100,['th']))
print(gen_sen_k(100,['ha']))
print(gen_sen_k(100,['s']))
print(gen_sen_k(100,['u']))

chak fis tadosire bly at atoruste zanthin viousilignd zee he  y anvedeaf ns s rntof ca y u llouchied
f n jintetha ped he uatewha e pealy grear  u wovareens g m be outnge g ulvens bun gy qun geill ra ry
rthallestingro vedind l ver é veintheayourololen f veanecays 30 chis gede zare  walewor qu k ve d y 
 xter n utaneals  ort clingy outshive mpith urathins did r 30 t l hen che yo  k xpesheacangor f abr 
r qu betyo vasa tuiecthen hes ang hig xisoo lle 30 qulyo be pawtr ine otandevindertolle ve s  linoti


### **Testing the Bigram(Interpolation) model**

In [14]:
kmodel.score('h','s'.split())

0.06022441275167785

# **Question 3 (5 points): Repeat the entire Question 2, but this time for a trigram character model. Report at least five output sentences starting with five arbitrary characters of your choice.**

### **Trigram Model(Laplace)**

In [15]:

# To create new_pad_list list
from nltk.lm.preprocessing import pad_both_ends
pad_en = []
for x in X_train:
         train_data = list(pad_both_ends(x, n=3))
         pad_en.append(train_data)
new_pad_list_k = []
for y in pad_en:
    for c in y:
        new_pad_list_k.append(c)

from nltk import trigrams

padded_trigram_list_k = []
tri_list_k = list(trigrams(new_pad_list_k))
padded_trigram_list_k.append(tri_list_k)

print(padded_trigram_list_k)

[[('<s>', '<s>', 'l'), ('<s>', 'l', 'i'), ('l', 'i', 's'), ('i', 's', 't'), ('s', 't', 'e'), ('t', 'e', 'n'), ('e', 'n', 'i'), ('n', 'i', 'n'), ('i', 'n', 'g'), ('n', 'g', '</s>'), ('g', '</s>', '</s>'), ('</s>', '</s>', '<s>'), ('</s>', '<s>', '<s>'), ('<s>', '<s>', 'a'), ('<s>', 'a', 'w'), ('a', 'w', 'a'), ('w', 'a', 'y'), ('a', 'y', '</s>'), ('y', '</s>', '</s>'), ('</s>', '</s>', '<s>'), ('</s>', '<s>', '<s>'), ('<s>', '<s>', 't'), ('<s>', 't', 'e'), ('t', 'e', 'n'), ('e', 'n', '</s>'), ('n', '</s>', '</s>'), ('</s>', '</s>', '<s>'), ('</s>', '<s>', '<s>'), ('<s>', '<s>', 't'), ('<s>', 't', 'a'), ('t', 'a', 'k'), ('a', 'k', 'e'), ('k', 'e', '</s>'), ('e', '</s>', '</s>'), ('</s>', '</s>', '<s>'), ('</s>', '<s>', '<s>'), ('<s>', '<s>', 'c'), ('<s>', 'c', 'h'), ('c', 'h', 'a'), ('h', 'a', 'r'), ('a', 'r', 'g'), ('r', 'g', 'e'), ('g', 'e', '</s>'), ('e', '</s>', '</s>'), ('</s>', '</s>', '<s>'), ('</s>', '<s>', '<s>'), ('<s>', '<s>', 'i'), ('<s>', 'i', 'm'), ('i', 'm', 'a'), ('m', 'a'

In [16]:
trigram_model=Laplace(3)
trigram_model.fit(padded_trigram_list_k,new_pad_list_k)

### **Testing the trigram model(Laplace)**

In [17]:
trigram_model.score('f',['u','h'])

0.03125

### **Sentence Generation(Trigram Laplace)**

In [18]:
def gen_sen_tri(num_words,text_seed):
    content=""
    for token in trigram_model.generate(num_words,text_seed,random_seed=None):
        if token=='<s>':
            token=' '
        elif token=='</s>':
            token=' '
        else:
            token=token
        content+=token
    return content

In [19]:
print(gen_sen_tri(100,text_seed=['s','s']))
print(gen_sen_tri(100,text_seed=['a','d']))
print(gen_sen_tri(100,text_seed=['i','u']))
print(gen_sen_tri(100,text_seed=['i','r']))
print(gen_sen_tri(100,text_seed=['g','o']))

    ining    the    n    ick    bunhattionte    whip    like    annes    youbt    or    phe    fort 
ds    gots    cus    thend    in    thoppinspatmes    ch    goled    ther    tor    sick    body    
ms    re    lettly    ente    aching    joy    ext    way    havess    negal    to    pirsuic    pla
entimputeravestachis    ind    try    nes    me    cons    s    ded    led    an    yough    bunnut 
lf    try    kip    gam    an    mucissist    bess    the    implays    regall    a    giver    vatc


### **Trigram model(Interpolation)**

In [20]:
trigram_model_k=KneserNeyInterpolated(3)
trigram_model_k.fit(padded_trigram_list_k,new_pad_list_k)

### **Testing the trigram model(Interpolated)**

In [21]:
trigram_model_k.score(['y','o','u'])

0.03125

### **Sentence Generation(Trigram Interpolated)**


In [22]:
def gen_sen_tri_k(num_words,text_seed):
    content=""
    for token in trigram_model_k.generate(num_words,text_seed,random_seed=None):
        if token=='<s>':
            token=' '
        elif token=='</s>':
            token=' '
        else:
            token=token
        content+=token
    return content

In [23]:
print(gen_sen_tri(100,text_seed=['s','s']))
print(gen_sen_tri(100,text_seed=['a','d']))
print(gen_sen_tri(100,text_seed=['i','u']))
print(gen_sen_tri(100,text_seed=['i','r']))
print(gen_sen_tri(100,text_seed=['g','o']))

    ply    whe    wintureadessicare    withemou    magest    and    hours    jithattly    an    sle 
    gare    a    ratmensinks    ifeaves    fute    you    insup    youfflishone    to    cludds    a
making    secoor    ar    lay    up    cardigh    futual    gookedio    bou    loorenclack    exam  
    cour    ger    rep    if    pler    past    frospy    withoto    ince    vals    movem    comed 
or    antery    argete    magetelasizzast    heater    ainter    ands    oatch    prom    the    pho
